In [11]:
import os
import numpy as np
import random
import string
from ase import Atoms
from ase.io import write


In [12]:
from utils.cs_gen import generate_atomic_positions

In [ ]:
from utils.initial_parameters import *
from utils.plots import plot_histogram
from utils.WC_initials import *

[]


In [ ]:
for sim in simulation_paths:
    for lattice_parameter in lattice_parameters:
        folder_path = os.path.join(sim, name_hea, f'lattice_par{lattice_parameter:.2f}')
        os.makedirs(folder_path, exist_ok=True)
# ------- atom creation part
        with open(folder_path + '/' + 'params.txt', 'w') as f:
            f.write(f'alat={lattice_parameter}\ncutoff={cuttof}\n')
        positions_dict = generate_atomic_positions(lattice_parameter, system_size, cs)
        
        positions = positions_dict[lattice_parameter]

        num_atoms = len(positions)

        atoms_per_type = num_atoms // num_types
        remainder = num_atoms % num_types

        symbols = []
        for atom in atom_types:
            symbols.extend([atom] * atoms_per_type)

        extra_atoms = random.sample(atom_types, remainder)
        symbols.extend(extra_atoms)

        random.shuffle(symbols)

        atoms = Atoms(symbols=symbols, positions=positions, cell=[lattice_parameter*system_size]*3, pbc=True)

        print(atoms)
        write(os.path.join(folder_path, f"{name_hea}.xyz"), atoms)
        write(os.path.join(folder_path, f"{name_hea}.data"), atoms, format='lammps-data')

# ------- id part
        random_id = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
        print(random_id)

        id_file_path = os.path.join(folder_path, f"{name_hea}_{random_id}id.txt")
        
        with open(id_file_path, 'w') as f:
            f.write(random_id)

        ids.append(random_id)


# ------- commands part
        command_copy_input_file = 'cp' + ' ' + input_file_path +input_files[0] + ' ' +folder_path
        command_copy = 'cp' + ' ' + executable_path +'wc_calc' + ' ' +folder_path
        command_copy1 = 'cp' + ' ' + executable_path +lmp_mpi + ' ' +folder_path
        command_wc_par = f'cd {folder_path} && ./wc_calc > WC-par_{name_hea}_{lattice_parameter:.2f}.txt'
        command_cp_pot = f'cp {potential_path + pot_files[0]}  {folder_path} '
        command_cp_pot1 = f'cp {potential_path + pot_files[1]}  {folder_path} '
        command_run_lammps_mpi = f'cd {folder_path} && {lmp_mpi} -in HfNbTaTiZr.in '

        os.system(command_copy_input_file)
        os.system(command_copy)
        os.system(command_copy1)
        os.system(command_cp_pot1)
        os.system(command_cp_pot)
        os.system(command_wc_par)
        os.system(command_run_lammps_mpi)
        parameters={}
        with open(folder_path +'/' + f'WC-par_{name_hea}_{lattice_parameter:.2f}.txt', 'r') as f:
            wc_results = f.readlines()

        for line in wc_results:
            if "=" in line:
                key, value = line.strip().split('=')
                parameters[key.strip()] = float(value.strip())

        parameters_HfHf.append(parameters['Alpha_Hf-Hf'])
        parameters_HfNb.append(parameters['Alpha_Hf-Nb'])
        parameters_HfTa.append(parameters['Alpha_Hf-Ta'])
        parameters_HfTi.append(parameters['Alpha_Hf-Ti'])
        parameters_HfZr.append(parameters['Alpha_Hf-Zr'])
        parameters_NbHf.append(parameters['Alpha_Nb-Hf'])
        parameters_NbNb.append(parameters['Alpha_Nb-Nb'])
        parameters_NbTa.append(parameters['Alpha_Nb-Ta'])
        parameters_NbTi.append(parameters['Alpha_Nb-Ti'])
        parameters_NbZr.append(parameters['Alpha_Nb-Zr'])
        parameters_TaHf.append(parameters['Alpha_Ta-Hf'])
        parameters_TaNb.append(parameters['Alpha_Ta-Nb'])
        parameters_TaTa.append(parameters['Alpha_Ta-Ta'])
        parameters_TaTi.append(parameters['Alpha_Ta-Ti'])
        parameters_TaZr.append(parameters['Alpha_Ta-Zr'])
        parameters_TiHf.append(parameters['Alpha_Ti-Hf'])
        parameters_TiNb.append(parameters['Alpha_Ti-Nb'])
        parameters_TiTa.append(parameters['Alpha_Ti-Ta'])
        parameters_TiTi.append(parameters['Alpha_Ti-Ti'])
        parameters_TiZr.append(parameters['Alpha_Ti-Zr'])
        parameters_ZrHf.append(parameters['Alpha_Zr-Hf'])
        parameters_ZrNb.append(parameters['Alpha_Zr-Nb'])
        parameters_ZrTa.append(parameters['Alpha_Zr-Ta'])
        parameters_ZrTi.append(parameters['Alpha_Zr-Ti'])
        parameters_ZrZr.append(parameters['Alpha_Zr-Zr'])


# ------- energies collection

        lammps_out = folder_path +'/' +'log.lammps'
        
        with open(lammps_out, 'r') as f:
            lines = f.readlines()
        initial_pe_2000 = lines[125].strip().split()
        initial_pe_1000 = lines[120].strip().split()



        print(initial_pe_1000)
        print(initial_pe_2000)


        energy_1000 = float(initial_pe_1000[4])
        energy_2000 = float(initial_pe_2000[4])


        energies_1000.append(energy_1000/16000)
        energies_2000.append(energy_2000/16000)

        print(energy_1000/16000)
        print(energy_2000/16000)

plot_histogram(parameters_HfHf, 'Alpha_Hf-Hf',ids=ids)
plot_histogram(parameters_HfNb, 'Alpha_Hf-Nb',ids=ids)
plot_histogram(parameters_NbTa, 'Alpha_Nb-Ta',ids=ids)